# Дисклеймер
Эту тетрадку нужно запускать в колабе или в vast.ai. Не мучатесь с установкой библиотек и с обучением на cpu.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks/nlp_hw')

In [3]:
!pip install tokenizers matplotlib sklearn

In [ ]:
# в vast ai или в последних версия jupyter может не работать автозаполнение, установка этой либы и перезагрука кернела помогает
# !pip install --upgrade jedi==0.17.2

# Транформеры для решения seq2seq задач

Seq2seq - наверное самая общая формальная постановка задачи в NLP. Нужно из произвольной последовательности получить какую-то другую последовательность. И в отличие от разметки последовательности (sequence labelling) не требуется, чтобы обе последовательности совпадали по длине. Даже стандартную задачу классификации можно решать как seq2seq - можно рассматривать метку класса как последовательность длинны 1.

А трансформеры - sota архитектура для seq2seq задач. Мы не будем подробно разбирать устройство транформеров, если вам интересно вы можете поразбираться вот с этими материалами:

Оригинальная статья (сложновато) - https://arxiv.org/pdf/1706.03762.pdf

https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/  
https://jalammar.github.io/illustrated-transformer/

https://www.youtube.com/watch?v=iDulhoQ2pro

https://www.youtube.com/watch?v=TQQlZhbC5ps

Самый известный туториал (на торче) - https://nlp.seas.harvard.edu/2018/04/03/attention.html



Трансформеры будут подробно разбираться на курсе глубокого обучения (по выбору) на втором курсе.

Пока просто попробуем обучать модель на задаче машинного перевода. Для таких задач лучше всего использовать предобученные модели, но если у вас будет какая-то специфичная seq2seq задача, то имеет смысл попробовать обучить трансформер с нуля и в этой тертрадке вам нужно будет поменять только часть с загрузкой данных. 

In [23]:
!pip install torch==1.8

     |████████████████████████████████| 735.5MB 21kB/s 
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.8.0 which is incompatible.
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.8.0 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102


In [ ]:
!unzip torch_weights.zip

Archive:  torch_weights.zip
   creating: torch_weights/
  inflating: torch_weights/tokenizer_ru  
  inflating: __MACOSX/torch_weights/._tokenizer_ru  
  inflating: torch_weights/model     
  inflating: __MACOSX/torch_weights/._model  
  inflating: torch_weights/tokenizer_en  
  inflating: __MACOSX/torch_weights/._tokenizer_en  


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
import nltk

In [38]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [39]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# в русскоязычных данных есть \xa0 вместо пробелов, он может некорректно обрабатываться токенизатором
#text = open('opus.en-ru-train.ru.txt').read().replace('\xa0', ' ')
#text = open('opus.en-ru-train.ru.txt').read().replace('\xa0', ' ')
#f = open('opus.en-ru-train.ru.txt', 'w')
#f.write(text)
#f.close()

Данные взяты вот отсюда - https://opus.nlpl.eu/opus-100.php (раздел с отдельными языковыми парами)

In [6]:
en_sents = open('opus.en-ru-train.en.txt').read().splitlines()
ru_sents = open('opus.en-ru-train.ru.txt').read().replace('\xa0', ' ').splitlines()

Пример перевода с английского на русский

In [7]:
en_sents[-1], ru_sents[-1]

('So what are you thinking?', 'Ну и что ты думаешь?')

Как обычно нам нужен токенизатор, а точнее даже 2, т.к. у нас два корпуса

In [ ]:
tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_en.train(files=["opus.en-ru-train.en.txt"], trainer=trainer_en)

tokenizer_ru = Tokenizer(BPE())
tokenizer_ru.pre_tokenizer = Whitespace()
trainer_ru = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_ru.train(files=["opus.en-ru-train.ru.txt"], trainer=trainer_ru)

### ВАЖНО!

Токенизатор - это неотъемлимая часть модели, поэтому не забывайте сохранять токенизатор вместе с моделью. Если вы забудете про это и переобучите токенизатор, то индексы токенов разойдутся и веса модели будут бесполезны. 

In [ ]:
# раскоментируйте эту ячейку при обучении токенизатора
# а потом снова закоментируйте чтобы при перезапуске не перезаписать токенизаторы
#tokenizer_en.save('tokenizer_en')
#tokenizer_ru.save('tokenizer_ru')

In [8]:
tokenizer_en = Tokenizer.from_file("torch_weights/tokenizer_en")
tokenizer_ru = Tokenizer.from_file("torch_weights/tokenizer_ru")

Переводим текст в индексы вот таким образом. В начало добавляем токен '[CLS]', а в конец '[SEP]'. Если вспомните занятие по языковому моделированию, то там мы добавляли "\<start>" и "\<end>" - cls и sep по сути тоже самое. Вы поймете почему именно cls и sep, а не start и end, если подробнее поразбираетесь с устройством трансформеров

In [9]:
def encode(text, tokenizer, max_len):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[SEP]')]

In [10]:
# важно следить чтобы индекс паддинга совпадал в токенизаторе с value в pad_sequences
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
PAD_IDX

3

In [121]:
# ограничимся длинной в 30 и 35 (разные чтобы показать что в seq2seq не нужна одинаковая длина)
max_len_en, max_len_ru = 30, 35

In [122]:
X_en = [encode(t, tokenizer_en, max_len_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, max_len_ru) for t in ru_sents]

In [123]:
# миллион примеров 
len(X_en), len(X_ru)

(1000000, 1000000)

Паддинг внутри класса для датасета. Еще обратите внимание, что тут не стоит параметр batch_first=True как раньше

В торче принято, что размерность батча идет в конце и пример кода с трансформером расчитан на это. Конечно можно поменять сам код модели, но это сложнее, чем просто изменить тензор с данными.

In [14]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts_en, texts_ru):
        self.texts_en = [torch.LongTensor(sent) for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, padding_value=PAD_IDX)
        
        self.texts_ru = [torch.LongTensor(sent) for sent in texts_ru]
        self.texts_ru = torch.nn.utils.rnn.pad_sequence(self.texts_ru, padding_value=PAD_IDX)

        self.length = len(texts_en)
    
    def __len__(self):
        return self.length

    def __getitem__(self, index):

        ids_en = self.texts_en[:, index]
        ids_ru = self.texts_ru[:, index]

        return ids_en, ids_ru

Разбиваем на трейн и тест

In [15]:
X_en_train, X_en_valid, X_ru_train, X_ru_valid = train_test_split(X_en, X_ru, test_size=0.05)

In [16]:
training_set = Dataset(X_en_train, X_ru_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=200, shuffle=True, )

In [17]:
valid_set = Dataset(X_en_valid, X_ru_valid)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=200, shuffle=True)

# Код трансформера

Дальше код модели, он взят вот отсюда (с небольшими изменениями) - https://pytorch.org/tutorials/beginner/transformer_tutorial.html

Там есть комментарии по каждому этапу

In [18]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 150):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size, 
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
#         print('pos inp')
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
#         print('pos dec')
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
#         print('pos out')
        x = self.generator(outs)
#         print('gen')
        return x

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
# During training, we need a subsequent word mask that will prevent model to look into the future words when making predictions. We will also need masks to hide source and target padding tokens. Below, let’s define a function that will take care of both.

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

Обратите внимание на то как мы подаем данные в модель

In [19]:
from time import time
def train(model, iterator, optimizer, criterion, print_every=500):
    
    epoch_loss = []
    ac = []
    
    model.train()  

    for i, (texts_en, texts_ru) in enumerate(iterator):
        texts_en = texts_en.T.to(DEVICE) # чтобы батч был в конце
        texts_ru = texts_ru.T.to(DEVICE) # чтобы батч был в конце
        
        # помимо текста в модель еще нужно передать целевую последовательность
        # но не полную а без 1 последнего элемента
        # а на выходе ожидаем, что модель сгенерирует этот недостающий элемент
        texts_ru_input = texts_ru[:-1, :]
        
        
        # в трансформерах нет циклов как в лстм 
        # каждый элемент связан с каждым через аттеншен
        # чтобы имитировать последовательную обработку
        # и чтобы не считать аттеншн с паддингом 
        # в трансформерах нужно считать много масок
        # подробнее про это по ссылкам выше
        (texts_en_mask, texts_ru_mask, 
        texts_en_padding_mask, texts_ru_padding_mask) = create_mask(texts_en, texts_ru_input)
        logits = model(texts_en, texts_ru_input, texts_en_mask, texts_ru_mask,
                       texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
        optimizer.zero_grad()
        
        # сравниваем выход из модели с целевой последовательностью уже с этим последним элементом
        texts_ru_out = texts_ru[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_ru_out.reshape(-1))
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
        
        if not (i+1) % print_every:
            print(f'Loss: {np.mean(epoch_loss)};')
        
    return np.mean(epoch_loss)


def evaluate(model, iterator, criterion):
    
    epoch_loss = []
    epoch_f1 = []
    
    model.eval()  
    with torch.no_grad():
        for i, (texts_en, texts_ru) in enumerate(iterator):
            texts_en = texts_en.T.to(DEVICE)
            texts_ru = texts_ru.T.to(DEVICE)

            texts_ru_input = texts_ru[:-1, :]

            (texts_en_mask, texts_ru_mask, 
            texts_en_padding_mask, texts_ru_padding_mask) = create_mask(texts_en, texts_ru_input)

            logits = model(texts_en, texts_ru_input, texts_en_mask, texts_ru_mask,
                           texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)

            
            texts_ru_out = texts_ru[1:, :]
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_ru_out.reshape(-1))
            epoch_loss.append(loss.item())
            
    return np.mean(epoch_loss)

In [20]:
torch.manual_seed(0)

EN_VOCAB_SIZE = tokenizer_en.get_vocab_size()
RU_VOCAB_SIZE = tokenizer_ru.get_vocab_size()

EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, EN_VOCAB_SIZE, RU_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
# torch.save(transformer, 'model')

In [22]:
transformer = torch.load('torch_weights/model', map_location=DEVICE)#.to(DEVICE)

In [ ]:
#torch.cuda.empty_cache()

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 100

losses = []

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train(transformer, training_generator, optimizer, loss_fn)
    end_time = timer()
    val_loss = evaluate(transformer, valid_generator, loss_fn)
    
    if not losses:
        print(f'First epoch - {val_loss}, saving model..')
        torch.save(transformer, 'model')
    
    elif val_loss < min(losses):
        print(f'Improved from {min(losses)} to {val_loss}, saving model..')
        torch.save(transformer, 'model')
    
    losses.append(val_loss)
        
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))

KeyboardInterrupt: ignored

In [23]:
def translate(text):

    input_ids = [tokenizer_en.token_to_id('[CLS]')] + tokenizer_en.encode(text).ids[:max_len_en] + [tokenizer_en.token_to_id('[SEP]')]
    output_ids = [tokenizer_ru.token_to_id('[CLS]')]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids)]).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

    (texts_en_mask, texts_ru_mask, 
    texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
    logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                   texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
    pred = logits.argmax(2).item()

    while pred not in [tokenizer_ru.token_to_id('[SEP]'), tokenizer_ru.token_to_id('[PAD]')]:
        output_ids.append(pred)
        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

        (texts_en_mask, texts_ru_mask, 
        texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
        logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                       texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
        pred = logits.argmax(2)[-1].item()

    return (' '.join([tokenizer_ru.id_to_token(i).replace('##', '') for i in output_ids[1:]]))

In [24]:
translate("Example")

'Пример'

In [25]:
translate('Can you translate that?')

'Ты можешь пере водить это ?'

In [149]:
def translate_2(text):

    #split text into sentences
    sents = nltk.tokenize.sent_tokenize(text)
    
    input_ids_all = []
    output_ids_all = []

    for sent in sents:
      input_ids = [tokenizer_en.token_to_id('[CLS]')] + tokenizer_en.encode(sent).ids[:max_len_en] + [tokenizer_en.token_to_id('[SEP]')]
      output_ids = [tokenizer_ru.token_to_id('[CLS]')]
      for i in input_ids:
        input_ids_all.append(i)
      for i in output_ids:
        output_ids_all.append(i)

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids_all)]).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids_all)]).to(DEVICE)
    
    (texts_en_mask, texts_ru_mask, texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
    logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                         texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)

    for logit in logits.argmax(2):
      pred = logit.item()
      
      while pred not in [tokenizer_ru.token_to_id('[SEP]'), tokenizer_ru.token_to_id('[PAD]')]:
          output_ids_all.append(pred)
          output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids_all)]).to(DEVICE)

          (texts_en_mask, texts_ru_mask, texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
          logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                              texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
          pred = logits.argmax(2)[-1].item()

    res = ' '.join([tokenizer_ru.id_to_token(i).replace('##', '') for i in output_ids_all])
    res = re.sub('\[CLS] ', '', res)

    return res

In [83]:
translate_2("Example")

'Пример'

In [84]:
translate_2('Can you translate that?')

'Ты можешь пере водить это ?'

In [85]:
translate_2('He told me i could eat his cake. It was a lie.')

'Он сказал мне , что я могу съесть торт . Он был сол гал .'

Здорово, что последнее предложение не перевелось дословно (хотя "был" остался)

Возьмем текст побольше

In [137]:
test_text = 'Journalists who were aboard the British warship at the time suggested that the confrontation with Russian forces was expected. One reporter noted that “increasingly hostile warnings were issued over the radio” by the Russian coast guard. Russia summoned the British ambassador, as well as the UK’s military attache in Moscow, to protest over the incident. '
test_text

'Journalists who were aboard the British warship at the time suggested that the confrontation with Russian forces was expected. One reporter noted that “increasingly hostile warnings were issued over the radio” by the Russian coast guard. Russia summoned the British ambassador, as well as the UK’s military attache in Moscow, to protest over the incident. '

In [138]:
translate_2(test_text)

'Жур нали сты , которые были на борту британ ского военного корабля , предложен ы , как ожидается , что противо стоя щая роль России . Жур нали сты , как ожидается , Жур'

Ну.. начало вроде ничего

In [147]:
tt = 'The latest training mission in the Mediterranean comes just two weeks after the Russian Navy launched large-scale military drills in the central Pacific Ocean, involving around 20 surface warships, submarines, and support vessels. This mission, which took place around 4,000km away from base, was aimed at practicing managing a large group of ships far from home.'
tt

'The latest training mission in the Mediterranean comes just two weeks after the Russian Navy launched large-scale military drills in the central Pacific Ocean, involving around 20 surface warships, submarines, and support vessels. This mission, which took place around 4,000km away from base, was aimed at practicing managing a large group of ships far from home.'

In [150]:
translate_2(tt)

'Последний курс обучения в Средизем но мор ском районе проходит через две недели после того , как в рамках российского В МС начали масштаб ные военные действия в центра льном районе , где было Последний'

А тут слова уже не так часто дробятся